# Grafici Coronavirus

Come utilizzare il Notebook:

- **Importare sempre i moduli (Blocco sotto)**
- **Prima volta**: se non si hanno i dati dei giorni passati -> eseguire **Step 0 e Step 2**
- Se si hanno i dati e si vogliono quelli giornalieri -> eseguire **Step 1 e Step 2**
- Se si vuole solo analizzare i grafici -> eseguire **Step 2**

Importiamo i moduli

In [ ]:
import datetime
import pandas as pd
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import shutil
pd.options.mode.chained_assignment = None

----------------------------------------------------------------------
----------------------------------------------------------------------

### Step 0 - Caricare dati antecedenti (SOLO PRIMA VOLTA)

La prima volta è necessario avere tutti i dati dei giorni precedenti (**non necessario dopo il primo caricamento**):

- Scaricare i dati dal sito della Protezione Civile: https://github.com/pcm-dpc/COVID-19)

In [ ]:
url = "https://github.com/pcm-dpc/COVID-19.git"
os.system("git clone " + url)
shutil.move("./COVID-19/dati-regioni","./")
shutil.move("./COVID-19/dati-andamento-nazionale","./")
os.remove("./dati-regioni/dpc-covid19-ita-regioni.csv")
os.remove("./dati-regioni/dpc-covid19-ita-regioni-latest.csv")
os.remove("./dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv")
os.remove("./dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale-latest.csv")

shutil.rmtree('./COVID-19')

In [ ]:
regions = ["Italia","Abruzzo","Basilicata","P.A. Bolzano","Calabria","Campania","Emilia-Romagna","Friuli Venezia Giulia","Lazio","Liguria","Lombardia","Marche","Molise","Piemonte","Puglia","Sardegna","Sicilia","Toscana","P.A. Trento","Umbria","Valle d'Aosta","Veneto"]

In [ ]:
dataframe_collection = {} 
for region in regions:
    print(region)
    dataframe_collection[region] = pd.DataFrame()
    if region == "Italia":
        for filename in os.listdir("./dati-andamento-nazionale"):
            if filename.endswith(".csv"):
                f = open("./dati-andamento-nazionale/" + filename)
                temp_df = pd.read_csv(f)
                temp_df= temp_df.loc[temp_df['stato'] == "ITA"]
                dataframe_collection[region] = dataframe_collection[region].append(temp_df,"\n")
                del temp_df
    else:
        for filename in os.listdir("./dati-regioni"):
            if filename.endswith(".csv"):
                f = open("./dati-regioni/" + filename)
                temp_df = pd.read_csv(f)
                temp_df= temp_df.loc[temp_df['denominazione_regione'] == region]
                dataframe_collection[region] = dataframe_collection[region].append(temp_df,"\n")
                del temp_df
    dataframe_collection[region] = dataframe_collection[region].sort_values(by=['data'], ignore_index=True)
    
    #Create new columns, to create other plots
    dataframe_collection[region]["nuovi_deceduti"] = dataframe_collection[region]["deceduti"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_guariti"] = dataframe_collection[region]["dimessi_guariti"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_ricoverati_con_sintomi"] = dataframe_collection[region]["ricoverati_con_sintomi"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_terapia_intensiva"] = dataframe_collection[region]["terapia_intensiva"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_isolamento_domiciliare"] = dataframe_collection[region]["isolamento_domiciliare"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_tamponi"] = dataframe_collection[region]["tamponi"].diff().fillna(0).astype(int)
   
    #Change date format
    dataframe_collection[region]["data"] = pd.to_datetime(dataframe_collection[region].data)
    dataframe_collection[region]["data"] = dataframe_collection[region]["data"].dt.strftime('%d/%m')

In [ ]:
for key in dataframe_collection.keys():    
    if not os.path.exists("./dati-regioni/cumulativi/"):
        os.makedirs("./dati-regioni/cumulativi/")
    if not os.path.exists("./dati-andamento-nazionale/cumulativi/"):
        os.makedirs("./dati-andamento-nazionale/cumulativi/")

    if key == "Italia":
        dataframe_collection[key].to_csv("./dati-andamento-nazionale/cumulativi/"+ key + ".csv", index=False)
    else:
        dataframe_collection[key].to_csv("./dati-regioni/cumulativi/"+ key + ".csv", index=False)

------------------------------------------
------------------------------------------

### Step 1 - Scaricare i dati giornalieri dalla Protezione Civile (ESEGUIRE SOLO DOPO LA PUBBLICAZIONE GIORNALIERA DEI DATI - dalle h19)

Scriviamo il csv giornaliero su disco

In [ ]:
now = datetime.datetime.now()
today_is=now.strftime("%Y%m%d")
regions = ["Italia","Abruzzo","Basilicata","P.A. Bolzano","Calabria","Campania","Emilia-Romagna","Friuli Venezia Giulia","Lazio","Liguria","Lombardia","Marche","Molise","Piemonte","Puglia","Sardegna","Sicilia","Toscana","P.A. Trento","Umbria","Valle d'Aosta","Veneto"]

In [ ]:
input_filename_regioni = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-" + today_is + ".csv"
output_filename_regioni = "./dati-regioni/dpc-covid19-ita-regioni-" + today_is + ".csv"
virus_today_regioni = pd.read_csv(input_filename_regioni)
virus_today_regioni.to_csv(output_filename_regioni,index=False)

input_filename_italia = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale-" + today_is + ".csv"
output_filename_italia = "./dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale-" + today_is + ".csv"
virus_today_italia = pd.read_csv(input_filename_italia)
virus_today_italia.to_csv(output_filename_italia,index=False)

Uniamo il dato giornaliero al file cumulativo, regione per regione:

In [ ]:
dataframe_collection = {}
for region in regions:
    if region == "Italia":
        today_collection = pd.DataFrame()
        dataframe_collection[region] = pd.read_csv("./dati-andamento-nazionale/cumulativi/" + region + ".csv")
        today_collection = virus_today_italia.loc[virus_today_italia['stato'] == "ITA"]   
        today_collection["data"] = pd.to_datetime(today_collection.data)
        today_collection["data"] = today_collection["data"].dt.strftime('%d/%m')
    else:
        today_collection = pd.DataFrame()
        dataframe_collection[region] = pd.read_csv("./dati-regioni/cumulativi/" + region + ".csv")
        today_collection = virus_today_regioni.loc[virus_today_regioni['denominazione_regione'] == region]
        today_collection["data"] = pd.to_datetime(today_collection.data)
        today_collection["data"] = today_collection["data"].dt.strftime('%d/%m')
    
    dataframe_collection[region]= dataframe_collection[region].append(today_collection,"\n")   
    #Create new columns, to create other plots
    dataframe_collection[region]["nuovi_deceduti"] = dataframe_collection[region]["deceduti"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_guariti"] = dataframe_collection[region]["dimessi_guariti"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_ricoverati_con_sintomi"] = dataframe_collection[region]["ricoverati_con_sintomi"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_terapia_intensiva"] = dataframe_collection[region]["terapia_intensiva"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_isolamento_domiciliare"] = dataframe_collection[region]["isolamento_domiciliare"].diff().fillna(0).astype(int)
    dataframe_collection[region]["nuovi_tamponi"] = dataframe_collection[region]["tamponi"].diff().fillna(0).astype(int)


In [ ]:
dataframe_collection["Marche"]

In [ ]:
for key in dataframe_collection.keys():
    if key == "Italia":
        dataframe_collection[key].to_csv("./dati-andamento-nazionale/cumulativi/"+ key + ".csv", index=False)
    else:
        dataframe_collection[key].to_csv("./dati-regioni/cumulativi/"+ key + ".csv", index=False)

### Step 2 - Fare i grafici (ESEGUIBILE SEMPRE)

In [ ]:
now = datetime.datetime.now()
show_time=now.strftime("%d/%m/%Y")
regions = ["Italia","Abruzzo","Basilicata","P.A. Bolzano","Calabria","Campania","Emilia-Romagna","Friuli Venezia Giulia","Lazio","Liguria","Lombardia","Marche","Molise","Piemonte","Puglia","Sardegna","Sicilia","Toscana","P.A. Trento","Umbria","Valle d'Aosta","Veneto"]

In [ ]:
dataframe_collection = {}
for region in regions:
    if region == "Italia":
        dataframe_collection[region] = pd.read_csv("./dati-andamento-nazionale/cumulativi/" + region + ".csv")
    else:
        dataframe_collection[region] = pd.read_csv("./dati-regioni/cumulativi/" + region + ".csv")

Creiamo qualche grafico:

In [ ]:
def incrementi_giornalieri(region): #Incrementi giornalieri nel numero di guariti, positivi e deceduti.
    #Plots
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_positivi"],
        mode='lines+markers',
        name='Nuovi Positivi',
        marker_color='orange',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_deceduti"],
        mode='lines+markers',
        name='Nuovi Deceduti',
        marker_color='black',
        marker_size=9
    ))

    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_guariti"],
        mode='lines+markers',
        name='Nuovi Guariti',
        marker_color='green',
        marker_size=9
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Sommario (Incrementi giornalieri)",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        xaxis=dict(dtick=5)
    )

    fig.show()

In [ ]:
def situazione_complessiva(region): #Situzione complessiva nel numero di guariti, attualmente positivi e deceduti
    #Plots
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["totale_casi"],
        mode='lines+markers',
        name='Totale Positivi',
        marker_color='red',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["totale_positivi"],
        mode='lines+markers',
        name='Attualmente Positivi',
        marker_color='orange',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["deceduti"],
        mode='lines+markers',
        name='Deceduti',
        marker_color='black',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["dimessi_guariti"],
        mode='lines+markers',
        name='Guariti',
        marker_color='green',
        marker_size=9
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Sommario (Incrementi giornalieri)",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        xaxis=dict(dtick=6),
        yaxis=dict(hoverformat='.f')
    )

    fig.show()

In [ ]:
def situazione_positivi(region):  #Evoluzione degli attualmente positivi (Tra ricoveri e domiciliati)
    #Plots
    fig = go.Figure()
    
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["totale_positivi"],
        mode='lines+markers',
        name='Totale Positivi (Somma)',
        hoverlabel = dict(namelength = -1),
        marker_color='red',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["terapia_intensiva"],
        mode='lines+markers',
        name='Terapia Intensiva',
        hoverlabel = dict(namelength = -1),
        marker_color='purple',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["ricoverati_con_sintomi"],
        mode='lines+markers',
        name='Ricoverati con sintomi',
        hoverlabel = dict(namelength = -1),
        marker_color='orange',
        marker_size=9
    ))
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["isolamento_domiciliare"],
        mode='lines+markers',
        name='Isolamento Domiciliare',
        hoverlabel = dict(namelength = -1),
        marker_color='blue',
        marker_size=9
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Situazione Positivi",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        hovermode="closest",
        xaxis=dict(dtick=6),
        yaxis=dict(hoverformat='.f')
    )

    fig.show()

In [ ]:
def carico_ospedaliero(region): #Variazione giornalieri nel numero di terapie intensive e ricoveri.
    #Plots
    fig = go.Figure()
    
    
    fig.add_trace(go.Bar(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_ricoverati_con_sintomi"],
        name='Nuovi Ricoverati con sintomi',
        hoverlabel = dict(namelength = -1),
        marker_color='orange',
    ))
    
    fig.add_trace(go.Bar(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_terapia_intensiva"],
        name='Nuovi Terapia Intensiva',
        hoverlabel = dict(namelength = -1),
        marker_color='red',
    ))
    
    fig.add_trace(go.Bar(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_isolamento_domiciliare"],
        name='Nuovi Isolamento Domiciliare',
        hoverlabel = dict(namelength = -1),
        marker_color='blue',
    ))

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Carico ospedaliero",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        xaxis_title="Data",
        yaxis_title="Persone",
        font=dict(
        size=12),
        hovermode="closest",
        xaxis=dict(dtick=6),
        yaxis=dict(hoverformat='.f')
    )

    fig.show()

In [ ]:
def tamponi_e_contagio(region): #Numero di nuovi positivi e nuovi tamponi con rapporto contagi/tamponi.

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(go.Bar(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_positivi"],
        name='Nuovi Positivi',
        hoverlabel = dict(namelength = -1),
        marker_color='red',
    ),
        secondary_y=False,   
    )
    
    fig.add_trace(go.Bar(
        x=dataframe_collection[region]["data"],
        y=dataframe_collection[region]["nuovi_tamponi"],
        name='Tamponi Giornalieri',
        hoverlabel = dict(namelength = -1),
        marker_color='grey',
    ),
        secondary_y=False,
    )
    
    fig.add_trace(go.Scatter(
        x=dataframe_collection[region]["data"],
        y=(dataframe_collection[region]["nuovi_positivi"].astype(float)/dataframe_collection[region]["nuovi_tamponi"])*100,
        mode="lines",
        name='Positivi a tampone',
        hoverlabel = dict(namelength = -1),
        marker_color='black',
    ),
        secondary_y=True,     
    )

    fig.update_layout(
        title={'text': region + " - " + show_time + " - Nuovi positivi e tamponi",'y':0.9,'x':0.5,'xanchor':'center','yanchor': 'top'},
        font=dict(
        size=12),
        hovermode="closest",
        xaxis=dict(dtick=6),
        yaxis=dict(hoverformat='.f')
    )
    
    # Set x-axis title
    fig.update_xaxes(title_text="Data")
    
    # Set y-axes titles
    fig.update_yaxes(title_text="Tamponi eseguiti e nuovi positivi", secondary_y=False)
    fig.update_yaxes(title_text="Rapporto nuovi contagi / tamponi", secondary_y=True)
    fig.update_yaxes(ticksuffix="%", secondary_y=True)
    fig.update_yaxes(hoverformat='.1f', secondary_y=True)


    fig.show()

Funzione per eseguire i plot, specificare regione all'interno della funzione che si vuole invocare:

Regioni disponibili:
- "Italia"
- "Abruzzo"
- "Basilicata"
- "P.A. Bolzano"
- "Calabria"
- "Campania"
- "Emilia-Romagna"
- "Friuli Venezia Giulia"
- "Lazio"
- "Liguria"
- "Lombardia"
- "Marche"
- "Molise"
- "Piemonte"
- "Puglia"
- "Sardegna"
- "Sicilia"
- "Toscana"
- "P.A. Trento"
- "Umbria"
- "Valle d'Aosta"
- "Veneto"

In [ ]:
tamponi_e_contagio("Marche")